In [4]:
#import libraries
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support, roc_auc_score


import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Concatenate, GlobalAveragePooling2D, Dense, Dropout,
    BatchNormalization, Conv2D, Lambda
)
from tensorflow.keras.applications import VGG16, DenseNet121
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Dataset Loading
classes = ['benign', 'malignant', 'normal']

def load_image_files(root_dir, classes, extensions=('.jpg', '.jpeg', '.png')):
    file_paths, labels = [], []
    for cls in classes:
        class_dir = os.path.join(root_dir, cls)
        if not os.path.exists(class_dir):
            print(f" Folder not found: {class_dir}")
            continue
        for root, _, files in os.walk(class_dir):
            for file in files:
                if file.lower().endswith(extensions):
                    file_paths.append(os.path.join(root, file))
                    labels.append(cls)
    return np.array(file_paths), np.array(labels)

root_dir = "/content/drive/MyDrive/Dataset"  #  update this path to your dataset
file_paths, labels = load_image_files(root_dir, classes)
file_paths, labels = shuffle(file_paths, labels, random_state=42)
print(f" Total images loaded: {len(file_paths)}")


# Data Generators
IMG_SIZE = (224, 224)
BATCH_SIZE = 16

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)
val_datagen = ImageDataGenerator(rescale=1./255)

def make_generator(datagen, X, y, batch_size=BATCH_SIZE, shuffle=True):
    df = pd.DataFrame({'filename': X, 'class': y})
    return datagen.flow_from_dataframe(
        df,
        x_col='filename',
        y_col='class',
        target_size=IMG_SIZE,
        class_mode='categorical',
        batch_size=batch_size,
        shuffle=shuffle
    )

# Hybrid Model with Lambda (resize + attention)
def create_hybrid_model(input_shape=(224,224,3), num_classes=3):
    inputs = Input(shape=input_shape)

    # Base models
    vgg = VGG16(weights='/content/drive/MyDrive/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False, input_tensor=inputs)
    densenet = DenseNet121(weights='/content/drive/MyDrive/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False, input_tensor=inputs)

    # Feature maps
    vgg_out = vgg.get_layer('block4_conv3').output
    densenet_out = densenet.get_layer('conv4_block24_concat').output

    # Resize DenseNet output to VGG feature map size
    densenet_out_resized = Lambda(lambda x: tf.image.resize(x, (28,28)))(densenet_out)

    # Channel reduction
    vgg_out = Conv2D(512, (1,1), activation='relu')(vgg_out)
    densenet_out_resized = Conv2D(512, (1,1), activation='relu')(densenet_out_resized)

    # Concatenate
    combined = Concatenate(axis=-1)([vgg_out, densenet_out_resized])

    # Attention mechanism
    attention = Conv2D(1, (1,1), activation='sigmoid')(combined)
    attended_features = Lambda(lambda x: x[0] * x[1])([combined, attention])

    # Head layers
    x = Conv2D(512, (3,3), padding='same', activation='relu')(attended_features)
    x = BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D((2,2))(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.6)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.4)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    return Model(inputs=inputs, outputs=outputs)

# 5-Fold Cross Validation Training
kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold = 1
metrics_summary = []

for train_idx, val_idx in kf.split(file_paths):
    print(f"\n🔹 Fold {fold} Training...")

    X_train, X_val = file_paths[train_idx], file_paths[val_idx]
    y_train, y_val = labels[train_idx], labels[val_idx]

    train_gen = make_generator(train_datagen, X_train, y_train)
    val_gen = make_generator(val_datagen, X_val, y_val, shuffle=False)

    model = create_hybrid_model(input_shape=(224,224,3), num_classes=len(classes))
    model.compile(
        optimizer=Adam(1e-4),
        loss='categorical_crossentropy',
        metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(),
                 tf.keras.metrics.AUC(multi_label=True)]
    )

    callbacks = [
        EarlyStopping(monitor='val_auc', patience=8, mode='max', restore_best_weights=True, verbose=1),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7, verbose=1),
        ModelCheckpoint(f'best_model_fold_{fold}.h5', monitor='val_auc', mode='max', save_best_only=True, verbose=1)
    ]

    model.fit(train_gen, validation_data=val_gen, epochs=30, callbacks=callbacks, verbose=1)

    # Predictions and metrics
    y_pred = model.predict(val_gen)
    y_true = np.array(val_gen.classes)
    y_pred_classes = np.argmax(y_pred, axis=1)

    cm = confusion_matrix(y_true, y_pred_classes)
    print(f"Confusion Matrix - Fold {fold}:\n{cm}")

    report = classification_report(y_true, y_pred_classes, target_names=classes, output_dict=True)
    acc = report['accuracy']
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred_classes, average='weighted')
    auc = roc_auc_score(tf.keras.utils.to_categorical(y_true, num_classes=len(classes)), y_pred, multi_class='ovr')

    metrics_summary.append([acc, precision, recall, f1, auc])
    fold += 1


# Final Average Metrics
metrics_summary = np.array(metrics_summary)
print("\n===== ✅ Final 5-Fold CV Results =====")
print(f"Accuracy: {metrics_summary[:,0].mean():.4f} ± {metrics_summary[:,0].std():.4f}")
print(f"Precision: {metrics_summary[:,1].mean():.4f} ± {metrics_summary[:,1].std():.4f}")
print(f"Recall: {metrics_summary[:,2].mean():.4f} ± {metrics_summary[:,2].std():.4f}")
print(f"F1 Score: {metrics_summary[:,3].mean():.4f} ± {metrics_summary[:,3].std():.4f}")
print(f"AUC: {metrics_summary[:,4].mean():.4f} ± {metrics_summary[:,4].std():.4f}")


 Total images loaded: 800

🔹 Fold 1 Training...
Found 640 validated image filenames belonging to 3 classes.
Found 160 validated image filenames belonging to 3 classes.


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.5974 - auc: 0.6602 - loss: 0.9118 - precision: 0.6473 - recall: 0.4939
Epoch 1: val_auc improved from -inf to 0.92549, saving model to best_model_fold_1.h5


40/40 ━━━━━━━━━━━━━━━━━━━━ 349s 4s/step - accuracy: 0.5998 - auc: 0.6637 - loss: 0.9067 - precision: 0.6498 - recall: 0.4969 - val_accuracy: 0.8625 - val_auc: 0.9255 - val_loss: 0.5993 - val_precision: 1.0000 - val_recall: 0.6438 - learning_rate: 1.0000e-04
Epoch 2/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 454ms/step - accuracy: 0.8219 - auc: 0.9165 - loss: 0.3630 - precision: 0.8473 - recall: 0.8032
Epoch 2: val_auc improved from 0.92549 to 0.92940, saving model to best_model_fold_1.h5


40/40 ━━━━━━━━━━━━━━━━━━━━ 22s 550ms/step - accuracy: 0.8220 - auc: 0.9165 - loss: 0.3629 - precision: 0.8475 - recall: 0.8033 - val_accuracy: 0.8313 - val_auc: 0.9294 - val_loss: 0.4860 - val_precision: 0.8897 - val_recall: 0.8062 - learning_rate: 1.0000e-04
Epoch 3/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 457ms/step - accuracy: 0.8784 - auc: 0.9366 - loss: 0.2964 - precision: 0.8805 - recall: 0.8726
Epoch 3: val_auc improved from 0.92940 to 0.96893, saving model to best_model_fold_1.h5


...

40/40 ━━━━━━━━━━━━━━━━━━━━ 31s 787ms/step - accuracy: 0.9890 - auc_4: 0.9824 - loss: 0.0374 - precision_4: 0.9890 - recall_4: 0.9890 - val_accuracy: 0.9500 - val_auc_4: 0.9952 - val_loss: 0.1721 - val_precision_4: 0.9500 - val_recall_4: 0.9500 - learning_rate: 6.2500e-06
Epoch 27/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 451ms/step - accuracy: 0.9960 - auc_4: 0.9999 - loss: 0.0129 - precision_4: 0.9960 - recall_4: 0.9960

40/40 ━━━━━━━━━━━━━━━━━━━━ 37s 686ms/step - accuracy: 0.9960 - auc_4: 0.9999 - loss: 0.0130 - precision_4: 0.9960 - recall_4: 0.9960 - val_accuracy: 0.9688 - val_auc_4: 0.9957 - val_loss: 0.1251 - val_precision_4: 0.9688 - val_recall_4: 0.9688 - learning_rate: 3.1250e-06
Epoch 28/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 465ms/step - accuracy: 0.9962 - auc_4: 1.0000 - loss: 0.0149 - precision_4: 0.9962 - recall_4: 0.9962

40/40 ━━━━━━━━━━━━━━━━━━━━ 29s 720ms/step - accuracy: 0.9961 - auc_4: 1.0000 - loss: 0.0149 - precision_4: 0.9961 - recall_4: 0.9961 - val_accuracy: 0.9688 - val_auc_4: 0.9958 - val_loss: 0.0958 - val_precision_4: 0.9688 - val_recall_4: 0.9688 - learning_rate: 3.1250e-06
Epoch 29/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 452ms/step - accuracy: 0.9991 - auc_4: 1.0000 - loss: 0.0059 - precision_4: 1.0000 - recall_4: 0.9991
Epoch 29: ReduceLROnPlateau reducing learning rate to 1.56249996052793e-06.


40/40 ━━━━━━━━━━━━━━━━━━━━ 31s 759ms/step - accuracy: 0.9991 - auc_4: 1.0000 - loss: 0.0059 - precision_4: 1.0000 - recall_4: 0.9991 - val_accuracy: 0.9688 - val_auc_4: 0.9956 - val_loss: 0.0986 - val_precision_4: 0.9688 - val_recall_4: 0.9688 - learning_rate: 3.1250e-06
Epoch 30/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 451ms/step - accuracy: 0.9971 - auc_4: 1.0000 - loss: 0.0108 - precision_4: 0.9971 - recall_4: 0.9971

40/40 ━━━━━━━━━━━━━━━━━━━━ 31s 769ms/step - accuracy: 0.9971 - auc_4: 1.0000 - loss: 0.0109 - precision_4: 0.9971 - recall_4: 0.9971 - val_accuracy: 0.9688 - val_auc_4: 0.9956 - val_loss: 0.0939 - val_precision_4: 0.9688 - val_recall_4: 0.9688 - learning_rate: 1.5625e-06
10/10 ━━━━━━━━━━━━━━━━━━━━ 12s 163ms/step
Confusion Matrix - Fold 5:
[[22  0  0]
 [ 0 74  0]
 [ 5  0 59]]

===== ✅ Final 5-Fold CV Results =====
Accuracy: 0.9650 ± 0.0109
Precision: 0.9674 ± 0.0108
Recall: 0.9650 ± 0.0109
F1 Score: 0.9656 ± 0.0108
AUC: 0.9914 ± 0.0060
